# RRTMGP _g_-Point Reduction With Cost Function Optimization

## Dependencies

The global paths and how this notebook uses them have been designed to rely as little as possible on any developer's environment. The aim is:

- use executable builds in E3SM Project Space on NERSC (this should be a viable option because everyone that logs into a NERSC machine will be using similar systems)
- write outputs to the user's home directory (`~`)
- import standard libraries (out-of-the-box Python and typical `pip` or `conda` installs like NumPy) from their NERSC locations
- import non-standard libraries (for this notebook, just `xarray`) from Rick Pernak's Anaconda environment

In [ ]:
# standard modules
import os, sys, shutil, subprocess, time, multiprocessing

# conda/pip installs; "standard" within NERSC JupyterHub
import netCDF4 as nc
import numpy as np

# path global variables
HOME = os.path.expanduser('~')
PROJECTDIR = '/project/projectdirs/e3sm/pernak18'
GPTHOME = '{}/g-point-reduction'.format(PROJECTDIR)
OPTPATH = '{}/k-distribution-opt'.format(GPTHOME)
LIBPATHS = [OPTPATH, 
            '{}/.local/'.format(os.path.expanduser('~pernak18')) + \
            'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages']
for path in LIBPATHS: sys.path.append(path)

# submodules from Robert Pincus/Ben Hillman
# Menno also has functions with the same name
# https://github.com/RobertPincus/k-distribution-opt/tree/brhillman/dev
# these are in OPTPATH
from combine_gpoints_fn import combine_gpoints_fn
import cost_function as CF

## RRTMGP Library and Executable Builds

First, we need to build the RTE+RRTMGP libraries and the executable that uses them -- `rrtmgp_garand_atmos` -- with the source code that is in the submodule directories `rte-rrtmgp` and ` k-distribution-opt`, respectively. This will be done in the top level directory of the clone for this repository, which has been defined as `GPTHOME` in the previous cell. To compile the `librrtmgp.a` and `librte.a` static libraries:

```
cd rte-rrtmgp/build
ln -s Makefile.conf.ifort Makefile.conf
make
```

And for the `rrtmgp_garand_atmos` executable, assuming the user has returned to the directory with this notebook in it:

```
cd k-distribution-opt/
git checkout nersc-notebook
export RRTMGP_ROOT="~/RRTMGP/g-point-reduction/rte-rrtmgp"
make
```

The absolute path for the executable will be used later in the notebook when the global variable `EXE` is defined.

## Function Definitions

Initially, functions were extracted from [Menno's `optimizer.py`](https://github.com/MennoVeerman/k-distribution-opt/blob/master/optimizer.py) and slightly modified. Currently, these are not used because we are starting with Ben's code. However, once we start incorporating Menno's cost function flexibility and diagnostic plotting, we may need his functions. The next cell contains functions that are compatible with Ben Hillman's [optimizerLW.py](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/optimizeLW.py).

In [ ]:
def pathCheck(path):
    """
    Determine if file exists. If not, throw an Assertion Exception
    """

    assert os.path.exists(path), 'Could not find {}'.format(path)

def make_name_variant(iterNum, gpts):
    """
    Return string of form iterNNN.GGG.GGG
       iterNum is a scalar int, gpts is list of ints of length 2
    """

    return('iter{0:03d}.{1[0]:03d}.{1[1]:03d}'.format(iterNum, gpts))

def trial_cost_function(iterNum, start_kdist_file, gpts, wts, 
                        ref_flux_file, cf_norm, 
                        inVars=['net_flux', 'heating_rate', 'band_flux_net'], 
                        inP=[0, 10000, 102000], 
                        exe='./rrtmgp_garand_atmos'):
    """
    Return the terms in the cost function (flux, heating rate, etc.) for a 
    proposed combination of two gpoints from an initial set.

    Inputs
        iterNum -- int, iteration number for documentation purposes
        start_kdist_file -- string, absorption coefficients netCDF
            used as an input into RRTMGP `exe`
        gpts -- list of 2 ints, g-point numbers
        wts -- list of 2 floats, weights associated with `gpts`
        ref_flux_file -- string, RRTMGP-formatted netCDF with reference 
            (likely LBLRTM) fluxes and heating rates in it
        cf_norm -- list of floats, normalization factors for each 
            cost function component
    
    Keywords
        inVars -- list of strings, netCDF variables used in cost function
        inP -- list of floats, pressure levels in Pa to be used in 
            cost function
        exe -- string, path to RRTMGP executable for Garand atmospheres

    Outputs
        error_components -- list of floats, normalized RMS error 
            of RRTMGP fluxes with respect to LBLRTM reference fluxes
    """

    trial_kdist_file = 'temp/coeffs/coefficients_{0}.nc'.format(
        make_name_variant(iterNum, gpts))
    trial_flux_file  = 'temp/fluxes/fluxes.all.{0}.nc'.format(
        make_name_variant(iterNum, gpts))

    # Make sure directories exist
    os.makedirs(os.path.dirname(trial_kdist_file), exist_ok=True)
    os.makedirs(os.path.dirname(trial_flux_file), exist_ok=True)

    # Combine the pair of g-points specified in gpts(2) with weights wts(2)
    combine_gpoints_fn(start_kdist_file, trial_kdist_file, gpts, wts)

    # Compute fluxes; note we copy the template flux file first because the 
    # driver overwrites fluxes in the original file rather than writing a 
    # new file
    shutil.copy2(ref_flux_file, trial_flux_file)
    subprocess.run([exe, trial_flux_file, trial_kdist_file], check=True)

    # Return the cost function components normalized by error at iteration 0
    error_components = CF.normCost(
        trial_flux_file, ref_flux_file, cf_norm, ncVars=inVars, levs=inP)
    assert all([np.isfinite(e) for e in error_components])
    return(error_components)

def writeCostNC(iteration, inCost, gCombine, inVars, costID):
    """
    Save terms of the cost function in a netCDF

    Input
        iteration -- int, g-point combination iteration number for 
          which cost function was calculated
        inCost -- list of cost function arrays for each variable 
          in inVars
        gCombine -- list of g-point combinations used; this is an
          n-combination-element list of 2-element lists that 
          contain the two g-points that are combined
        inVars -- string list of netCDF variable names used in 
          cost function calculation; should have the same number 
          of elements as gCombine does arrays
        costID -- int, cost function index
        normID -- int, error type or normalization ID

    Output
        netCDF file that follows the convention
        `data/cost_function_terms.lw.iterIII.costCC.normNN.nc`
            III -- 0-padded 3-digit iteration number
            CC -- 0-padded 2-digit cost function ID
            NN -- 0-padded 2-digit error/normalization ID
    """

    ncFile = 'data/cost_function_terms.lw.iter' + \
        '{:03d}.cost{:02d}.nc'.format(iteration, costID)

    # have to transpose the arrays for netCDF processing to work
    inCostT = np.array(inCost).T

    with nc.Dataset(ncFile, 'w') as dataOUT:
        dataOUT.createDimension('pair', 2)
        dataOUT.createDimension('combination', len(gCombine))

        # loop over cost function components
        for comp, sVar in zip(inCostT, inVars):
            ncVar = dataOUT.createVariable(sVar, 'f4', ('combination'))
            ncVar[:] = np.array(comp)

        # weighted sum of all cost function components
        totalCost = dataOUT.createVariable(
            'total_cost_fn', 'f4', ('combination'))
        totalCost[:] = [CF.total_cost(cost) for cost in inCost]

        # g-point combinations used
        gPtsOut = dataOUT.createVariable(
            'Gpt_pair', 'f4', ('combination', 'pair'))

## Additional Global Variables

Whether the _k_-distribution optimization is done in the longwave or shortwave domain is specified with `f_thermal` (type `bool`). This is important for filename specifications. Forcing has not been implemented in this notebook, so it should **always** be set to `False`.

In [ ]:
THERMAL = True
FORCING = False

# FRANGE is a list of indices that represent forcing scenarios, 
# so it only depends on whether LW is selected, and there will
# always be 6 forcing scenarios (in addition to no forcing)
FRANGE = range(1+6*(THERMAL and FORCING))

DOMAIN = 'lw' if THERMAL else 'sw'
DOMAINL = 'longwave' if THERMAL else 'shortwave'

# newest reference file provided by Robert (with `record` dimension)
FILELBLRTM = '{}/inputs/g-point-reduce/lblrtm-{}'.format(PROJECTDIR, DOMAIN)
FILELBLRTM += '-flux-inputs-outputs-garandANDpreind.nc'

EXE = '{}/g-point-reduction/k-distribution-opt/rrtmgp_garand_atmos'.format(
    PROJECTDIR)

paths = [FILELBLRTM, EXE]
for path in paths: pathCheck(path)


Note that the executable is the [FORTRAN-based](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/rrtmgp_garand_atmos.F90) `rrtmgp_garand_atmos` rather than the [C++-based](https://github.com/MennoVeerman/k-distribution-opt/blob/master/test_garand.cpp) `test_garand`. The latter was yielding `Segmentation Faults` at runtime on the NERSC `cori` machine and thus was not producing the results that were needed for the rest of the optimization. Because of these errors, we pursue the "Hillman" method in the rest of this notebook, while attempting to fold in the enhancements from Menno, which include cost function flexibility and diagnostics plotting. Hillman and Pincus also have more transparent cost function calculation, which we will try to extend to the Menno cost functions.

## Cost Function Selection

Trials will typically differ in their cost function definition and normalization factor, both of which are defined in this cell. We define the cost function not only by what parameters for which errors are calculated, but also where in the atmosphere they are calculated. Any new cost function definitions should adhere to the convention in following cell -- an additional `elif` clause should be added to the `ICOST` conditional, and it should define what `varsCF` is. Strings that are permitted in `varsCF` are:

- band_flux_dn
- band_flux_net
- band_flux_up
- band_heating_rate
- flux_dn
- flux_net
- flux_up
- heating_rate

Other variables (i.e., all that are in the netCDF) are available, but are likely not of interest for the cost function and may cause the optimization code to crash. While `ICOST` and `varsCF` could just be defined and replaced with every trial, doing it this way documents what was done for future reference.

Each trial should have an associated pressure level array or list -- `pCF` -- that should be used with the variables and a pressure string -- `pStr` -- that is used with the output directories. Pressures need to be in Pa for consistency with RRTMGP convention, and the cost function code finds the pressures from `p_lev` (reference pressures from LBLRTM run -- Menno grabs the no-forcing scenario pressure profile for the first Garand atmosphere) that are closest to the user-provided `pCF` values, then averages over all pressure levels and Garand profiles.

Currently, a limitation of the code used in this notebook is that cost function components are computed for one pressure range at a time. Variables in Menno's code each had corresponding regions because he had a different Python function for each cost function, which was difficult (but still possible) to generalize. We will come back to this if necessary.

We can also use this block if we decide to do other kinds of normalization (AKA error type), but currently we are normalizing with the RRTGMP-LBLRTM RMS error before any reduction in _g_-points is performed. Any change in normalization should be documented in this notebook and in the output directories.

In [ ]:
# Global variable definition of the cost function type
ICOST = 0

# define netCDF variables in the optimization
# ICOST 1:8 emulate Menno's cost functions; just starting this
# define pTrop and split p_lev and use trop and strat for pCF
if ICOST == 0: varsCF = ['flux_net', 'band_flux_net', 'heating_rate']
elif ICOST == 2: varsCF = ['flux_net']
else: print('INVALID COST FUNCTION')

# where in the profile should we minimize the 
# no forcing, all reference pressure levels, and first Garand
# probably should be more robust and extract 
# more than just the first Garand profile
p_lev = nc.Dataset(FILELBLRTM).variables['p_lev'][0,:,0]

# pressure at tropopause in Pa
pTrop = np.exp(4.6) * 10

pCF = np.where(p_lev >= pTrop)[0]
pStr = 'troposphere'

costStr = '{:s}_cost{:02d}_norm'.format(pStr, ICOST)

print('Variables in cost function: {}'.format(', '.join(varsCF)))

## Path Assignments

Paths depend on the parent directory paths, cost function specification, and whether we are examining the longwave or shortware domains. So if any of those parameters are changed, this cell needs to be re-run.

In [ ]:
# where to save coeff and temporary flux files (originally `dpath`)
outDatPath = '{}/RRTMGP/g-point-reduction/intermediate_files'.format(HOME)
if not os.path.isdir(outDatPath): os.makedirs(outDatPath)

# RRTMGP coefficient file
coeffInit = 'rrtmgp-data-lw-g256-2018-12-04.nc' if THERMAL else \
    'rrtmgp-data-sw-g224-2018-12-04.nc'
dirCoeffInit = '{}/rte-rrtmgp/rrtmgp/data/'.format(GPTHOME)
pathCheck(dirCoeffInit)

# Directory where new fluxes and coefficient files are stored.
dirRes  = "{}/results/{}.cost{:02d}".format(outDatPath, DOMAIN, ICOST)
if not os.path.isdir(dirRes): os.makedirs(dirRes)

## main()

### Runtime Variables Definition, Extract Information from LBLRTM Reference Results

The number of _g_-points can be extracted from the LBL netCDF as well. The weights associated with each _g_-point are the same for each band, so we effectively produce an `nGpt`x`nBnds` array, then flatten it to a 1-D vector.

In [ ]:
# Initial settings.
# Todo: generalize, read values from coefficient file
# Number of bands
nBnds = len(nc.Dataset(FILELBLRTM).variables['band'][:])

# Number of G-points in each band.
nGptsPerBandOrg = 16

# Number of G-points
nGpt = nBnds * nGptsPerBandOrg

# optimization iterations
nOptIt = 210
nOptIt = 3

# Band ID for each G-point
bandID = range(1, nBnds+1)
bandID = np.repeat(bandID, nGptsPerBandOrg)

# G-point weights (same for all bands)
# expand weights for one band to the rest of the bands with np.tile
# so weights are an (nGpt x nBnds)-element vector
wgtBnd = [0.1527534276, 0.1491729617, 0.1420961469, 0.1316886544, 
         0.1181945205, 0.1019300893, 0.0832767040, 0.0626720116, 
         0.0424925000, 0.0046269894, 0.0038279891, 0.0030260086, 
         0.0022199750, 0.0014140010, 0.0005330000, 0.0000750000]
wt = np.tile(wgtBnd, nBnds)

### RRTMGP File Staging

In [ ]:
# Prepare input file
"""
file_rrtmgp_input = "{}/input/rte_rrtmgp_input_{}.nc".format(
    dirData, DOMAIN)
os.makedirs(os.path.dirname(file_rrtmgp_input), exist_ok=True)

# Create output directory
file_rrtmgp_output = "{}/fluxes/rte_rrtmgp_output_{}.nc".format(
    dirData, DOMAIN)
os.makedirs(os.path.dirname(file_rrtmgp_output), exist_ok=True)

# copy original k-dist file
shutil.copy2('{}/{}'.format(dirCoeffInit, coeffInit), 
             '{}/{}'.format(dirData, coeffInit))
"""

### Initial RRTMGP Fluxes (With Original _k_-distribution)/Reference Fluxes

Stage some more files into a working directory (where the model is run over many iterations), then perform an initial run of RRTMGP over all Garand atmospheres. `returncode` of 0 means success. Be leery of other return codes.

In [ ]:
os.chdir(dirRes)

# copy k-distribution and profile information files to working dir
# we are using Robert's LBL reference file for the latter
# https://github.com/RobertPincus/k-distribution-opt/blob/master/lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc
coeffNC = 'coefficients_{}.nc'.format(DOMAIN)
shutil.copy2('{}/{}'.format(dirCoeffInit, coeffInit), coeffNC)
inNC = 'rte_rrtmgp_input-output_0.nc'
shutil.copy2(FILELBLRTM, './{}'.format(inNC))

# run Robert's version of `test_garand`
args = [EXE, inNC, coeffNC]
status = subprocess.run(args)
if status.returncode != 0:
    print('WARNING! {} did not complete.'.format(' '.join(args)))
    print('This is likely because of a segmentation fault.')
else:
    print('Initial RRTMGP fluxes calculation complete')

Save the RRTMGP fluxes from initial run so that they can be used to determine the normalization factor that is use in the optimization iterations.

In [ ]:
# File Staging
file_rrtmgp_ref = '{}/fluxes/reference_kdist_fluxes.nc'.format(dirRes)
if not os.path.isdir('{}/fluxes'.format(dirRes)):
    os.makedirs('{}/fluxes'.format(dirRes))
os.rename(inNC, file_rrtmgp_ref)

# normalization factor
cf_norm = CF.costFuncComp(
    file_rrtmgp_ref, FILELBLRTM, ncVars=varsCF, levs=pCF)

### Greedy Optimization of Cost Function

#### Menno version

[Menno's Optimizer](https://github.com/MennoVeerman/k-distribution-opt/blob/master/optimizer.py) and Ben Hillman's (Ben separates into [LW](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/optimizeLW.py) and [SW](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/optimizeSW.py) optimizers) are pretty similar. Menno experiments with many more cost functions and plots intermediate results more often, but there seems to be some inconsistencies between the input files and perhaps executables that are used. For example, Menno uses `ncecat` to add a `record` dimension to the netCDF files, but Robert's and Ben's `rrtmgp_garand_atmos` expects it at runtime. Since we used `rrtmgp_garand_atmos`, we will use Ben's approach to optimization.

#### Hillman Version

Now the computationally-extensive part. First, some additional file staging is performed, most important being the gathering of the initial _k_-distribution for the first iteration. Then, a loop over a user-specified number of iterations (`nOptIt`) is executed, where the following steps are followed:

1. _k_-distribution is defined
2. determine all _g_-point and associated weight combinations for the iteration
3. cost function for all possible _g_-point and weight pairs is calculated -- these pairs are distributed over many CPU threads; also calculate new coefficients and write them to their own _k_-distribution file
4. the pair that minimized the errors is determined
5. a new _k_-distribution, which was generated in step 3., is defined for the next iteration
6. clean up of unnecessary files
7. summary of optimization is printed to notebook
8. modify weights according the optimal _g_-point combination, to be used in step 2. of the next iteration

In [ ]:
# still in dirRes
os.makedirs('data', exist_ok=True)

# On first iteration, coefficients file is the original one
shutil.copy2('{}/{}'.format(dirCoeffInit, coeffInit), 
             'data/{}'.format(coeffInit))
coeffPrev = str(coeffInit)

print(f'Combining g-point pairs {nOptIt} times...'); sys.stdout.flush()

for iMain in range(1, nOptIt):
    # Which coefficient file to use? 
    coeffIter = 'data/{}'.format(coeffPrev)
    pathCheck(coeffIter)

    # Create list of all adjacent g-point and weight pairs in each band
    gpt_list = [[x, x+1] for x in range(1, nGpt) if 
                bandID[x-1] == bandID[x]]
    wgt_list = [(wt[gpt_pair[0]-1], wt[gpt_pair[1]-1]) for 
                gpt_pair in gpt_list]

    # Compute error terms for each combination of adjacent g-points pairs
    # parallelize the calculations over all CPUs
    # THIS NEEDS TO BE ADJUSTED FOR NERSC! cutting the total nCores in half
    with multiprocessing.Pool(multiprocessing.cpu_count() // 2) as pool:
        # separate processes, each with their own arguments
        results = [pool.apply_async(trial_cost_function, 
                                   args=(iMain, coeffIter, gpt_pair, 
                                         wgt_pair, FILELBLRTM, cf_norm, 
                                         varsCF, pCF, EXE))
                   for gpt_pair, wgt_pair in zip(gpt_list, wgt_list)]
        cfn_list = [r.get() for r in results]

    # Greedy optimization
    # Of all the g-point combinations in this iteration, 
    # which had the smallest error?
    winner = np.argmin([CF.total_cost(x) for x in cfn_list])

    # Set the new coefficent file for the next iteration.
    coeffPrev = 'coefficients_{0}.nc'.format(
        make_name_variant(iMain, gpt_list[winner]))
    shutil.copy2('temp/coeffs/{}'.format(coeffPrev), 
                 'data/{}'.format(coeffPrev))

    # Remove temporary files
    for d in ['temp/coeffs', 'temp/fluxes']:
        for f in os.listdir(d): os.remove(os.path.join(d, f))

    g1out = int(gpt_list[winner][0])
    g2out = int(gpt_list[winner][1])

    # print to standard output the g-point combination that minimized error
    print('For iteration {0:03d}, '.format(iMain), end='')
    print('combining g-points {:03d} '.format(gpt_list[winner][0]), end='')
    print('and {:03d} '.format(gpt_list[winner][1]), end='')
    print('in band {:02d}'.format(bandID[g1out-1]))
    print()

    # summarize values that were optimized
    for iVar, var in enumerate(varsCF):
        print('{:7s}: {:9.8f}'.format(var, cfn_list[winner][iVar].values))

    print('{:10s}: {:9.8f}'.format(
        'Total Cost', CF.total_cost(cfn_list[winner])))
    print('New coefficient file: {}'.format(coeffPrev))

    # For the next iteration, modify the weights and the array that contains
    # the number of G-points in each band. Next iteration will be over nGpts-1
    wt[g1out-1] = wt[g1out-1] + wt[g2out-1]
    wt = np.delete(wt, g2out-1)
    bandID = np.delete(bandID, g2out-1)
    nGpt -= 1

    # write a netCDF for this iteration
    writeCostNC(iMain, cfn_list, gpt_list, varsCF, ICOST)